## Task 5

In [1]:

unigrams = {}
bigrams = {}

tags_by_words = {}

unigrams_tags = {}
bigrams_tags = {}



for line in open("../resources/supertags.txt"):
    word, tag = line.split()
    tags_by_words[word] = tag
    
    
    
print("got tags")    
    
for line in open("../poleval_corpus/poleval_2grams.txt"):
    count, fst, snd = line.split()
    
    count = int(count)
    
    if count < 2:
        continue
        
    
    if fst not in bigrams:
        bigrams[fst] = {}
        
    bigrams[fst][snd] = count
    
    if fst not in tags_by_words:
        fst_tag = "^"+fst[-3:]
    else:
        fst_tag = tags_by_words[fst]
        
    if snd not in tags_by_words:
        snd_tag = "^"+snd[-3:]
    else:
        snd_tag = tags_by_words[snd]
    
    if fst_tag not in bigrams_tags:
        bigrams_tags[fst_tag] = {}
        
    bigrams_tags[fst_tag][snd_tag] = bigrams_tags[fst_tag].get(snd_tag, 0) + 1
    
    if fst is not "<BOS>":
        unigrams[fst] = unigrams.get(fst, 0) + 1
        unigrams_tags[fst_tag] = unigrams_tags.get(fst_tag, 0) + 1
    
    if snd is not "<EOS>":
        unigrams[snd] = unigrams.get(snd, 0) + 1
        unigrams_tags[snd_tag] = unigrams_tags.get(snd_tag, 0) + 1
    
#    if snd == ",":
#        print(fst, snd, count)
#        print(fst_tag, snd_tag, count)
#       print(unigrams[snd_tag])
    
print("read")
    
for word in bigrams:
     bigrams[word]["#SUM#"] = sum(bigrams[word].values())

for tag in bigrams_tags:
     bigrams_tags[tag]["#SUM#"] = sum(bigrams_tags[tag].values())


unigrams["#SUM#"] = sum(unigrams.values())
unigrams_tags["#SUM#"] = unigrams["#SUM#"]


print("done")


got tags
read
done


In [2]:

def get_n_best(word, k, metric):
    """
       Returns k best words by metrics 
    """
    
    best = []
    for word2 in unigrams.keys():
        if word2 == "#SUM#":
            continue
        best.append((metric(word, word2), word2))
        best = sorted(best, reverse=True)[:10]
    return best
        



In [3]:
import math

def ppmi(word1, word2):
    if word1 not in bigrams:
        return 0
    if word2 not in bigrams[word1]:
        return 0 ## przyblizam przez log_2(1/2) - wygladzanie laplacea
    return max(0, math.log2( bigrams[word1][word2]*unigrams["#SUM#"]/(bigrams[word1]["#SUM#"]*unigrams[word2])))


In [4]:
def poisson_stirling_approximation(w1, w2):
    if w1 not in bigrams:
        return 0
    if w2 not in bigrams[w1]:
        return 0
    
    return bigrams[w1][w2]*(math.log2(bigrams[w1][w2])-math.log2(unigrams[w1]*unigrams[w2]/len(unigrams))-1)



In [15]:
def grammar_words_ppmi(w1, w2):
    if w1 not in bigrams:
        return 0
    if w2 not in bigrams[w1]:
        return 0
    
    
    if w1 not in tags_by_words:
        w1_tag = "^"+w1[-3:]
    else:
        w1_tag = tags_by_words[w1]
        
    if w2 not in tags_by_words:
        w2_tag = "^"+w2[-3:]
    else:
        w2_tag = tags_by_words[w2]
        
    return max(0, math.log2( bigrams[w1][w2]*unigrams["#SUM#"]/(bigrams[w1]["#SUM#"]*unigrams[w2]))) + \
           max(0, 0.5*math.log2( bigrams_tags[w1_tag][w2_tag]*unigrams_tags["#SUM#"]/(bigrams_tags[w1_tag]["#SUM#"]*unigrams_tags[w2_tag])))



In [16]:
words = ["krowa", "w", "piekna", "cześć", "i", "chyba", "dzien", "ostatnia", "pan", "san"]
methods = [ppmi, poisson_stirling_approximation, grammar_words_ppmi]

for word in words:
    d = {}
    for method in methods:
        best = get_n_best(word, 10, method)
        d[method] = [w for _, w in best]
    print(word)
    for i in range(10):
        print("\t\t".join([d[method][i] for method in methods]))
    print()






krowa
uprawiała		uprawiała		uprawiała
matylda		matylda		szkocka
szkocka		morska		morska
vs.		znosi		matylda
morska		szkocka		vs.
znosi		daje		przestanie
przestanie		vs.		znosi
produkuje		przestanie		produkuje
rasy		produkuje		rasy
daje		rasy		daje

w
oparciu		tym		oparciu
odniesieniu		związku		odniesieniu
odróżnieniu		polsce		odróżnieniu
przeciwieństwie		ramach		przeciwieństwie
porównaniu		zakresie		porównaniu
domesday		latach		domesday
zeszłym		tej		zeszłym
narciarstwie		sprawie		zremisowanym
zremisowanym		przypadku		kijowie-bykowni
ministerstwie		dniu		niedługim

piekna
panienko		pogoda		sciezka
sciezka		kobieta		panienko
madzia		panienko		madzia
mloda		suknia		mloda
suka		sciezka		plaza
pogoda		muzyka		mowi
suknia		madzia		suka
mowi		sprawa		pogoda
plaza		mloda		suknia
bramka		suka		działka

cześć
dipterologa		nazwano		dipterologa
rosyjskiegocaraaleksandra		św.		hemipterologa
hemipterologa		króla		rosyjskiegocaraaleksandra
jarmily		świętego		oddawaną
oddawaną		niemieckiego		szulgi
t